[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zincware/supercharge/blob/main/examples/example01.ipynb)

In [ ]:
!pip install supercharge

In [1]:
import supercharge
import time
import random
import pathlib

In [2]:
class Node(supercharge.Base):
    def __init__(self):
        self._run_time = []
        self._run_start_time = None

        self.random_number = None
        self._random_number_cache = None
        self._file = pathlib.Path('number.json')

    @supercharge.Charge
    def run(self):
        """Function to be timed"""
        raise NotImplementedError
    @run.enter
    def pre_run(self):
        self._run_start_time = time.time()
    @run.exit
    def post_run(self):
        execution_time = time.time() - self._run_start_time
        print(f"Elapsed time: {execution_time:.3f} s")
        self._run_time.append(execution_time)

    @supercharge.Charge
    def create_random_number(self):
        """Have a hidden save method"""
        self.random_number = random.randrange(0, 100)

    @create_random_number.enter
    def _cache_number(self):
        """Cache the old number"""
        self._random_number_cache = self.random_number

    @create_random_number.exit
    def _save_number(self):
        """Save the number to file if it changed"""
        if self._random_number_cache != self.random_number:
            self._file.write_text(f"{self.random_number}")

    @classmethod
    def load(cls):
        instance = cls()
        instance.random_number = int(instance._file.read_text())
        return instance

In [3]:
class MyNode(Node):
    def run(self):
        time.sleep(1)

my_node = MyNode()
my_node.run()

Elapsed time: 1.011 s


In [4]:
my_node.create_random_number()
my_node.random_number

53

In [5]:
MyNode.load().random_number

53